In [21]:
import random
import numpy as np
from scipy.stats import multivariate_normal, norm
from scipy.optimize import minimize
import pandas as pd

In [2]:
###### FLEMING'S FUNCTION MODIFIED
def compute_meta_conf_serialdependence(xp, a, sigma_act, sigma_conf, rho):
    
    dhat = np.array([-1, 1])
    mu_x_xp_dhat = np.zeros((2, len(xp)))
    var_x_xp_dhat = np.zeros(len(xp))
    rho_vec = np.full(len(xp), rho)
    sigA_vec = np.full(len(xp), sigma_act)
    sigP_vec = np.full(len(xp), sigma_conf)
    
    Tol = 10e-4

    for dhati in range(2):
        dhat_vec = np.full(len(xp), dhat[dhati])
        
        mu_x_xp_dhat[dhati, :] = dhat_vec + (sigA_vec / sigP_vec) * rho_vec * (xp - dhat_vec)
        var_x_xp_dhat = (1 - rho_vec**2) * sigA_vec**2
        
        if a == 1:
            p_a_dhat_xp = 1 - norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        else:
            p_a_dhat_xp = norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        
        lik_d = norm.pdf(xp, dhat_vec, sigP_vec)
        
        if dhati == 0:
            p_a_dhat_xp_full = p_a_dhat_xp
            lik_d_full = lik_d
        else:
            p_a_dhat_xp_full = np.vstack((p_a_dhat_xp_full, p_a_dhat_xp))
            lik_d_full = np.vstack((lik_d_full, lik_d))
    
    # manage probability
    p_a_dhat_xp_full = np.clip(p_a_dhat_xp_full, Tol, None)
    lik_d_full = np.clip(lik_d_full, Tol, None)
    
    lik_d_full = lik_d_full / np.sum(lik_d_full, axis=0, keepdims=True)
    p_dhat_xp_a = p_a_dhat_xp_full * lik_d_full
    p_dhat_xp_a = p_dhat_xp_a / np.sum(p_dhat_xp_a, axis=0, keepdims=True)
    
    # Conf = p(a=d)
    if a == 1:
        conf = p_dhat_xp_a[1, :]
    else:
        conf = p_dhat_xp_a[0, :]
    
    return conf

### Load the data

In [3]:
df = pd.read_csv('data_Mazancieux_2018.csv')
df.head()

,Subj_idx,Stimulus,Response,Confidence,RT_decision,RT_confidence,Task
0,1,2,2,4,6.848,2.217,VP
1,1,2,2,6,1.879,2.250,VP
2,1,2,2,7,4.022,1.182,VP
3,1,2,2,6,1.106,1.803,VP
4,1,2,2,6,1.611,1.254,VP


In [ ]:
# Standardized the variable
# df_vp_std_f = np.std(df_vp['Confidence'],ddof=1)
# df_vp_mean_f = np.mean(df_vp['Confidence'])
# df_vp.loc[:, 'Confidence_standardized'] = (df_vp['Confidence'] - df_vp_mean_f) / df_vp_std_f

# df_vp_std_r_f = np.std(df_vp['Response'],ddof=1)
# df_vp_mean_r_f = np.mean(df_vp['Response'])
# df_vp.loc[:, 'Response_standardized'] = (df_vp['Response'] - df_vp_mean_r_f) / df_vp_std_r_f

# # check that it has mean 0 and sd 1
# df_vp_std_conf = np.std(df_vp['Confidence_standardized'],ddof=1)
# df_vp_mean_conf = np.mean(df_vp['Confidence_standardized'])


In [ ]:
### covarianza between confidence and response
# Extraer las columnas 'Confidence' y 'Response'
# confidence = df_vp['Confidence_standardized']
# response = df_vp['Response_standardized']

# # Calcular la covarianza usando np.cov
# covariance_matrix = np.cov(confidence, response, ddof=1)
# covariance = covariance_matrix[0, 1]

# covariance

In [ ]:
# d = df_vp.loc[:,'Stimulus_transformed'].values


In [24]:
### Function to run the model, input: d (stimuli), output: confidence with and without serial dependence

def model_run(d, a):

    theta = [1/6, 2/6, 3/6, 4/6, 5/6] # [0/6, 1/6, 2/6, 3/6, 4/6, 5/6, 6/6] I delete two values because it was very easy o very difficult
    sigmaAct = 1  # 1
    sigmaConf = 1 # 1
    rho = 0.5 # 0.5
    bigSigma = np.array([[sigmaAct**2, rho * sigmaAct * sigmaConf], [rho * sigmaAct * sigmaConf, sigmaConf**2]])

    # serial dependence weight. alpha = 3 and beta = 7 gives a E[p] of 0.3 given the ecuation alpha/(alpha+beta)
    serial_dependence_weight_alpha = 3 
    serial_dependence_weight_beta = 7

    N = len(d) # 1000  # N trials

    xa = np.empty(N)
    xp = np.empty(N)
    # xp_serialDependence = np.full( N, 111.0)
    # d = df_vp.loc[:,'Stimulus_transformed'].values # np.empty(N)
    # a = np.empty(N)
    secondOrder_mean_cor = np.empty(N)
    secondOrder_mean_cor_serialDependence =  np.full( N, 111.0)
    # last_xp = np.empty(N)
    first_trial = True

    for i in range(N):
        current_theta = random.choice(theta) 
        
        # d[i] = 1 if np.random.rand() > 0.5 else -1
        
        r = multivariate_normal.rvs(mean=[d[i] * current_theta, d[i] * current_theta], cov=bigSigma)
        
        xa[i] = r[0]
        xp[i] = r[1]
        
        # if xa[i] > 0:
        #     a[i] = 1
        #     flip_a = 1
        # else:
        #     a[i] = -1
        #     flip_a = 0

        flip_a = a[i]

        secondOrder_mean_cor[i] = compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
        
        if first_trial == False:
            p_serial_dependence = np.random.beta(serial_dependence_weight_alpha, serial_dependence_weight_beta, 1)[0]
            secondOrder_mean_cor_serialDependence[i] = (
                p_serial_dependence * secondOrder_mean_cor_serialDependence[i-1] + 
                (1-p_serial_dependence) * compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
            )
        else:
            secondOrder_mean_cor_serialDependence[i] = compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
    
        first_trial = False

    # drop first value
    secondOrder_mean_cor_adj = secondOrder_mean_cor[1:]
    secondOrder_mean_cor_serialDependence_adj = secondOrder_mean_cor_serialDependence[1:]


    return secondOrder_mean_cor_adj, secondOrder_mean_cor_serialDependence_adj
        


### Fit the model confidence to the Confidence without serial dependence

In [25]:
## choose a task
df_vp = df[df['Task'] == 'VP'].copy()

## preproccess the stimulus
df_vp.loc[:,'Stimulus_transformed'] = df_vp['Stimulus'].replace({1:-1,2:1})
df_vp.loc[:,'Response_transformed'] = df_vp['Response'].replace({1:0,2:1})

## initialize two list two save the results
all_secondOrder_mean_cor = []
all_secondOrder_mean_cor_serialDependence = []

n_participants = len(df_vp['Subj_idx'].unique())

for n_p in range(n_participants+1):
    
    df_vp_participant = df_vp[df_vp['Subj_idx'] == n_p] 
    d = df_vp_participant['Stimulus_transformed'].values
    a = df_vp_participant['Response_transformed'].values
    secondOrder_mean_cor, secondOrder_mean_cor_serialDependence = model_run(d,a)

    # save results
    all_secondOrder_mean_cor.extend(secondOrder_mean_cor)
    all_secondOrder_mean_cor_serialDependence.extend(secondOrder_mean_cor_serialDependence)

## drop the first row of each participant (the function model_run do the same)
df_vp_cleaned = df_vp.groupby('Subj_idx', group_keys=False).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

## from list to array
all_secondOrder_mean_cor = np.array(all_secondOrder_mean_cor)
all_secondOrder_mean_cor_serialDependence = np.array(all_secondOrder_mean_cor_serialDependence)

## from array to df
df_results = pd.DataFrame({
    'secondOrder_mean_cor': all_secondOrder_mean_cor,
    'secondOrder_mean_cor_serialDependence': all_secondOrder_mean_cor_serialDependence
})

## concatenate results
df_vp_final = pd.concat([df_vp_cleaned.reset_index(drop=True), df_results.reset_index(drop=True)], axis=1)


C:\Users\marcosembon\AppData\Local\Temp\ipykernel_17796\3371121176.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_vp_cleaned = df_vp.groupby('Subj_idx').apply(lambda x: x.iloc[1:]).reset_index(level=0, drop=True).reset_index(drop=True)


In [26]:
df_vp_final['Confidence_0to1'] = df_vp_final['Confidence']/10
df_vp_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7059 entries, 0 to 7058
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Subj_idx                               7059 non-null   int64  
 1   Stimulus                               7059 non-null   int64  
 2   Response                               7059 non-null   int64  
 3   Confidence                             7059 non-null   int64  
 4   RT_decision                            7059 non-null   float64
 5   RT_confidence                          7059 non-null   float64
 6   Task                                   7059 non-null   object 
 7   Stimulus_transformed                   7059 non-null   int64  
 8   Response_transformed                   7059 non-null   int64  
 9   secondOrder_mean_cor                   7059 non-null   float64
 10  secondOrder_mean_cor_serialDependence  7059 non-null   float64
 11  Conf

### Fit the model confidence to the Confidence with serial dependence

### Simulate several participants?